### Disney Dataset Creation (BS4)

#### Scrape & clean a list of disney wiki pages to create dataset to analyze information
#### One box

In [10]:
from bs4 import BeautifulSoup as bs
import requests 

In [2]:
page=requests.get('https://en.wikipedia.org/wiki/Toy_Story_3')
soup=bs(page.content)

In [3]:
# test scrape 
""""
table=soup.find('table',class_="infobox vevent")
for data in soup.find_all('table',class_="infobox vevent"):
    money=data.find('th',scope_='row')
    print(money)
    
for x in table.find_all('tr'):
    money=x.find('th')
    for k in x.find_all('td'):
        print(k.get_text())"""

'"\ntable=soup.find(\'table\',class_="infobox vevent")\nfor data in soup.find_all(\'table\',class_="infobox vevent"):\n    money=data.find(\'th\',scope_=\'row\')\n    print(money)\n    \nfor x in table.find_all(\'tr\'):\n    money=x.find(\'th\')\n    for k in x.find_all(\'td\'):\n        print(k.get_text())'

In [4]:
#Scraping table
info_box=soup.find(class_='infobox vevent')

info_rows=info_box.find_all('tr')

for x in info_rows:
    print(x.prettify())

<tr>
 <th class="summary" colspan="2" style="text-align:center;font-size:125%;font-weight:bold;font-size:110%;font-style:italic;">
  Toy Story 3
 </th>
</tr>

<tr>
 <td colspan="2" style="text-align:center">
  <a class="image" href="/wiki/File:Toy_Story_3_poster.jpg" title="All of the toys packed close together, holding up a large numeral '3', with Buzz, who is putting a friendly arm around Woody's shoulder, and Woody holding the top of the 3.">
   <img alt="All of the toys packed close together, holding up a large numeral '3', with Buzz, who is putting a friendly arm around Woody's shoulder, and Woody holding the top of the 3." class="thumbborder" data-file-height="326" data-file-width="220" decoding="async" height="326" src="//upload.wikimedia.org/wikipedia/en/6/69/Toy_Story_3_poster.jpg" width="220"/>
  </a>
  <div style="font-size:95%;padding:0.35em 0.35em 0.25em;line-height:1.25em;">
   Theatrical release poster
  </div>
 </td>
</tr>

<tr>
 <th scope="row" style="white-space:nowra

In [5]:
#Bulding dictionares

movie_info={}

def get_content_value(row_data):
    if row_data.find("li"):
        return [li.get_text(" ",strip=True).replace("\xa0"," ") for li in row_data.find_all('li')]
    else :
        return row_data.get_text(" ",strip=True).replace("\xa0"," ")


for index, row in enumerate(info_rows):  #funkcja enumerate po to aby automatycznie tworzyc indeksy
    if index == 0:
        movie_info['title']=row.find('th').get_text(" ",strip=True)
        
    elif index == 1:
        continue
    else:
        content_key=row.find('th').get_text(" ",strip=True)
        content_value=get_content_value(row.find('td'))
        movie_info[content_key]=content_value

        
movie_info

{'title': 'Toy Story 3',
 'Directed by': 'Lee Unkrich',
 'Produced by': 'Darla K. Anderson',
 'Screenplay by': 'Michael Arndt',
 'Story by': ['John Lasseter', 'Andrew Stanton', 'Lee Unkrich'],
 'Starring': ['Tom Hanks',
  'Tim Allen',
  'Joan Cusack',
  'Don Rickles',
  'Wallace Shawn',
  'John Ratzenberger',
  'Estelle Harris',
  'Blake Clark',
  'Jeff Pidgeon',
  'Ned Beatty',
  'Michael Keaton',
  'Jodi Benson',
  'John Morris'],
 'Music by': 'Randy Newman',
 'Cinematography': ['Jeremy Lasky', 'Kim White'],
 'Edited by': 'Ken Schretzmann',
 'Production company': ['Walt Disney Pictures', 'Pixar Animation Studios'],
 'Distributed by': 'Walt Disney Studios Motion Pictures',
 'Release date': ['June 12, 2010 ( 2010-06-12 ) ( Taormina Film Fest )',
  'June 18, 2010 ( 2010-06-18 ) (United States)'],
 'Running time': '103 minutes [1]',
 'Country': 'United States',
 'Language': 'English',
 'Budget': '$200 million [1]',
 'Box office': '$1.067 billion [1]'}

#### Scraping all box for all disney movies

In [6]:
page=requests.get('https://en.wikipedia.org/wiki/List_of_Walt_Disney_Pictures_films')
soup=bs(page.content)

In [8]:
movies=soup.select('.wikitable.sortable i')
movies[0]

<i><a href="/wiki/Academy_Award_Review_of_Walt_Disney_Cartoons" title="Academy Award Review of Walt Disney Cartoons">Academy Award Review of Walt Disney Cartoons</a></i>

In [9]:
def get_content_value(row_data):
    if row_data.find("li"):
        return [li.get_text(" ",strip=True).replace("\xa0"," ") for li in row_data.find_all('li')]
    else :
        return row_data.get_text(" ",strip=True).replace("\xa0"," ")


def get_info_box(url):
    
    page=requests.get(url)
    soup=bs(page.content)
    
    info_box=soup.find(class_='infobox vevent')
    info_rows=info_box.find_all('tr')

    movie_info={}
    for index, row in enumerate(info_rows):  #funkcja enumerate po to aby automatycznie tworzyc indeksy
        if index == 0:
            movie_info['title']=row.find('th').get_text(" ",strip=True)

        elif index == 1:
            continue
        else:
            content_key=row.find('th').get_text(" ",strip=True)
            content_value=get_content_value(row.find('td'))
            movie_info[content_key]=content_value
    return movie_info

In [ ]:
page=requests.get('https://en.wikipedia.org/wiki/List_of_Walt_Disney_Pictures_films')
soup=bs(page.content)
movies=soup.select('.wikitable.sortable i a')

base_path="https://en.wikipedia.org/"

movie_info_list=[]

for index, movie in enumerate(movies):
    try:
        relative_path=movie['href']
        full_path=base_path + relative_path
        title=movie['title']

        movie_info_list.append(get_info_box(full_path))


    except Exception as e:
        print(movie.get_text())
        print(e)        

#### Save/Reload Movie Data in json file

In [38]:
import json

def save_data(title, data):
    with open(title,'w',encoding='utf-8') as f:
        json.dumb(data,f,enscure_ascii=False, indent=2 )

In [39]:
import json 

def load_data(title):
    with open(title,encoding='utf-8') as f:
        return json.load(f)

In [12]:
#save_data('disney_data.json',movie_info_list)

## Clean Data

In [7]:
movies_data=load_data('disney_data2.json')

In [4]:
movies_data

[{'title': 'Academy Award Review of Walt Disney Cartoons',
  'Production company': 'Walt Disney Productions',
  'Release date': ['May 19, 1937 ( 1937-05-19 )'],
  'Running time': '41 minutes (74 minutes 1966 release)',
  'Country': 'United States',
  'Language': 'English',
  'Box office': '$45.472'},
 {'title': 'Snow White and the Seven Dwarfs',
  'Directed by': ['David Hand (supervising)',
   'William Cottrell',
   'Wilfred Jackson',
   'Larry Morey',
   'Perce Pearce',
   'Ben Sharpsteen'],
  'Produced by': 'Walt Disney',
  'Written by': ['Ted Sears',
   'Richard Creedon',
   'Otto Englander',
   'Dick Rickard',
   'Earl Hurd',
   'Merrill De Maris',
   'Dorothy Ann Blank',
   'Webb Smith'],
  'Based on': 'Snow White by The Brothers Grimm',
  'Starring': ['Adriana Caselotti',
   'Lucille La Verne',
   'Harry Stockwell',
   'Roy Atwell',
   'Pinto Colvig',
   'Otis Harlan',
   'Scotty Mattraw',
   'Billy Gilbert',
   'Eddie Collins',
   'Moroni Olsen',
   'Stuart Buchanan'],
  'Music 

##### Substask
- Celan up references [1] 
- Convert running time into a intnger
- Convert dates into datetime object
- Split up the long strings -  done
- Convert budget and box office to numbers

In [1]:
#Celan up references [1], [2] etc / doubled film premiere

In [2]:
#Split up the long strings 

In [8]:
def get_content_value(row_data):
    if row_data.find("li"):
        return [li.get_text(" ",strip=True).replace("\xa0"," ") for li in row_data.find_all('li')]
    elif row_data.find('br'): # tworzenie lsty dla  aktorów wystepujacych w filmie, tam gdzie tego nie było / split up long string
        return [text for text in row_data.stripped_strings]        
    else :
        return row_data.get_text(" ",strip=True).replace("\xa0"," ")

def clean_tags(soup): # usuwanie [1],[2] etc oraz zbugowanych podwojnych dat wydania filmu
    for tag in soup.find_all(["sup","span"]):
        tag.decompose()    
    

def get_info_box(url):
    
    page=requests.get(url)
    soup=bs(page.content)
    
    info_box=soup.find(class_='infobox vevent')
    info_rows=info_box.find_all('tr')
    
    clean_tags(soup)
    
    movie_info={}
    for index, row in enumerate(info_rows):  #funkcja enumerate po to aby automatycznie tworzyc indeksy
        if index == 0:
            movie_info['title']=row.find('th').get_text(" ",strip=True)

        else:
            header=row.find('th')
            if header:
                content_key=row.find('th').get_text(" ",strip=True)
                content_value=get_content_value(row.find('td'))
                movie_info[content_key]=content_value
    return movie_info

In [15]:
get_info_box('https://en.wikipedia.org/wiki/Toy_Story_3')

{'title': 'Toy Story 3',
 'Directed by': 'Lee Unkrich',
 'Produced by': 'Darla K. Anderson',
 'Screenplay by': 'Michael Arndt',
 'Story by': ['John Lasseter', 'Andrew Stanton', 'Lee Unkrich'],
 'Starring': ['Tom Hanks',
  'Tim Allen',
  'Joan Cusack',
  'Don Rickles',
  'Wallace Shawn',
  'John Ratzenberger',
  'Estelle Harris',
  'Blake Clark',
  'Jeff Pidgeon',
  'Ned Beatty',
  'Michael Keaton',
  'Jodi Benson',
  'John Morris'],
 'Music by': 'Randy Newman',
 'Cinematography': ['Jeremy Lasky', 'Kim White'],
 'Edited by': 'Ken Schretzmann',
 'Production company': ['Walt Disney Pictures', 'Pixar Animation Studios'],
 'Distributed by': 'Walt Disney Studios Motion Pictures',
 'Release date': ['June 12, 2010 ( Taormina Film Fest )',
  'June 18, 2010 (United States)'],
 'Running time': '103 minutes',
 'Country': 'United States',
 'Language': 'English',
 'Budget': '$200 million',
 'Box office': '$1.067 billion'}

In [ ]:
#Convert running time into a integer

In [20]:
[movie.get('Running time','N/A') for movie in movies_data]

['41 minutes (74 minutes 1966 release)',
 '83 minutes',
 '88 minutes',
 '126 minutes',
 '74 minutes',
 '64 minutes',
 '70 minutes',
 '42 minutes',
 '65 min.',
 '71 minutes',
 '75 minutes',
 '94 minutes',
 '73 minutes',
 '75 minutes',
 '82 minutes',
 '68 minutes',
 '74 minutes',
 '96 minutes',
 '75 minutes',
 '84 minutes',
 '77 minutes',
 '92 minutes',
 '69 minutes',
 '81 minutes',
 ['60 minutes (VHS version)', '71 minutes (original)'],
 '127 minutes',
 '92 minutes',
 '76 minutes',
 '75 minutes',
 '73 minutes',
 '85 minutes',
 '81 minutes',
 '70 minutes',
 '90 min.',
 '80 minutes',
 '75 minutes',
 '83 minutes',
 '83 minutes',
 '72 minutes',
 '97 minutes',
 '75 minutes',
 '104 minutes',
 '93 minutes',
 '105 minutes',
 '95 minutes',
 '97 minutes',
 '134 minutes',
 '69 minutes',
 '92 minutes',
 '126 minutes',
 '79 minutes',
 '97 minutes',
 '128 minutes',
 '74 minutes',
 '91 minutes',
 '105 minutes',
 '98 minutes',
 '130 minutes',
 '89 min.',
 '93 minutes',
 '67 minutes',
 '98 minutes',
 '1

In [9]:
" 85 minutes"
def minute_to_intiger(running_time):
    if running_time == 'N/A':
        return None
    
    if isinstance(running_time,list):
        return int(running_time[0].split(' ')[0])     
    else:
        return int(running_time.split(' ')[0])
    
for movie in movies_data:
    movie['Running time (int)']=minute_to_intiger(movie.get('Running time','N/A'))

In [27]:
[movie.get('Running time (int)','N/A') for movie in movies_data]

[41,
 83,
 88,
 126,
 74,
 64,
 70,
 42,
 65,
 71,
 75,
 94,
 73,
 75,
 82,
 68,
 74,
 96,
 75,
 84,
 77,
 92,
 69,
 81,
 60,
 127,
 92,
 76,
 75,
 73,
 85,
 81,
 70,
 90,
 80,
 75,
 83,
 83,
 72,
 97,
 75,
 104,
 93,
 105,
 95,
 97,
 134,
 69,
 92,
 126,
 79,
 97,
 128,
 74,
 91,
 105,
 98,
 130,
 89,
 93,
 67,
 98,
 100,
 118,
 103,
 110,
 80,
 79,
 91,
 91,
 97,
 118,
 139,
 92,
 131,
 87,
 116,
 93,
 110,
 110,
 131,
 101,
 108,
 84,
 78,
 75,
 164,
 106,
 110,
 99,
 113,
 108,
 112,
 93,
 91,
 93,
 100,
 100,
 79,
 96,
 113,
 89,
 118,
 92,
 88,
 92,
 87,
 93,
 93,
 93,
 90,
 83,
 96,
 88,
 89,
 91,
 93,
 92,
 97,
 100,
 100,
 89,
 91,
 112,
 115,
 95,
 91,
 95,
 104,
 74,
 48,
 77,
 104,
 128,
 101,
 94,
 104,
 90,
 100,
 88,
 93,
 98,
 100,
 112,
 84,
 98,
 97,
 114,
 96,
 100,
 109,
 83,
 90,
 107,
 96,
 103,
 91,
 95,
 105,
 113,
 80,
 101,
 89,
 74,
 90,
 89,
 110,
 74,
 93,
 84,
 83,
 69,
 77,
 107,
 93,
 88,
 108,
 84,
 121,
 89,
 104,
 90,
 86,
 84,
 108,
 107,
 96,
 98,
 

In [28]:
#Convert budget and box office to numbers

In [13]:
import re

number=r"\d+(,\d{3})*\.*\d*"
amount = r"thousand|million|billion"
value_re = rf'\${number}'
word_re = rf'\${number}(-|\sto\s|–)?({number})?\s({amount})'

def word_to_value(word):
    value_dict = {'thousand':1000,'million':1000000,'billion':1000000000}
    return value_dict[word]


def parse_word_syntax(string):
    value_string=re.search(number,string).group()
    value = float(value_string.replace(',', ''))
    word = re.search(amount, string,flags=re.I).group().lower()
    word_value= word_to_value(word)
    return value * word_value


def parse_value_syntax(string):
    value_string=re.search(number,string).group()
    value=float(value_string.replace(',',''))
    return value

def money_conversion(money):
    if money =='N/A':
        return None
    if isinstance(money,list):
        money=money[0]

    word_syntax = re.search(word_re,money,flags=re.I)
    value_syntax = re.search(value_re,money)
    if word_syntax:
        return parse_word_syntax(word_syntax.group())
    elif value_syntax:
        return parse_value_syntax(value_syntax.group())

    else:
        return None

In [14]:
for movie in movies_data:
    movie['Budget (float)']=money_conversion(movie.get('Budget','N/A'))
    movie['Box office (float)']=money_conversion(movie.get('Box office','N/A'))  

In [15]:
movies_data[-40]

{'title': 'A Wrinkle in Time',
 'Directed by': 'Ava DuVernay',
 'Produced by': ['Jim Whitaker', 'Catherine Hand'],
 'Screenplay by': ['Jennifer Lee', 'Jeff Stockwell'],
 'Based on': ['A Wrinkle in Time', 'by', "Madeleine L'Engle"],
 'Starring': ['Oprah Winfrey',
  'Reese Witherspoon',
  'Mindy Kaling',
  'Levi Miller',
  'Storm Reid',
  'Gugu Mbatha-Raw',
  'Michael Peña',
  'Zach Galifianakis',
  'Chris Pine'],
 'Music by': 'Ramin Djawadi',
 'Cinematography': 'Tobias A. Schliessler',
 'Edited by': 'Spencer Averick',
 'Production companies': ['Walt Disney Pictures', 'Whitaker Entertainment'],
 'Distributed by': ['Walt Disney Studios', 'Motion Pictures'],
 'Release date': ['February 26, 2018 ( El Capitan Theatre )',
  'March 9, 2018 (United States)'],
 'Running time': '109 minutes',
 'Country': 'United States',
 'Language': 'English',
 'Budget': '$100–130 million',
 'Box office': '$133.4 million',
 'Running time (int)': 109,
 'Budget (float)': 100000000.0,
 'Box office (float)': 1334000

In [16]:
#Convert dates into datetime object

In [18]:
print([movie.get('Release date','N/A') for movie in movies_data])

[['May 19, 1937'], ['December 21, 1937 ( Carthay Circle Theatre , Los Angeles , CA )', 'February 4, 1938 (United States)'], ['February 7, 1940 ( Center Theatre )', 'February 23, 1940 (United States)'], ['November 13, 1940'], ['June 20, 1941'], ['October 23, 1941 (New York City)', 'October 31, 1941 (U.S.)'], ['August 9, 1942 (World Premiere-London)', 'August 13, 1942 (Premiere-New York City)', 'August 21, 1942 (U.S.)'], ['August 24, 1942 (World Premiere-Rio de Janeiro)', 'February 6, 1943 (U.S. Premiere-Boston)', 'February 19, 1943 (U.S.)'], ['July 17, 1943'], ['December 21, 1944 (Mexico City)', 'February 3, 1945 (US)'], ['April 20, 1946 (New York City premiere)', 'August 15, 1946 (U.S.)'], ['November 12, 1946 (Premiere: Atlanta, Georgia)', 'November 20, 1946'], ['September 27, 1947'], 'May 27, 1948', ['November 29, 1948 (Chicago, Illinois)', 'January 19, 1949 (Indianapolis, Indiana)'], ['October 5, 1949'], ['February 15, 1950 (Boston)', 'March 4, 1950 (United States)'], ['June 22, 1950

In [19]:
from datetime import datetime

In [32]:
dates=[movie.get('Release date','N/A') for movie in movies_data]

def clean_date(date):
    return date.split("(")[0].strip()

def date_conversion(date):
    if isinstance(date,list):
        date=date[0]
    if date =="N/A":
        return None
    date_str=clean_date(date)

    fmts=["%B %d, %Y","%d %B, %Y"]
    for fmt in fmts:
        try:
            return datetime.strptime(date_str,fmt)
        except:
            pass
    return None       

1937-05-19 00:00:00
1937-12-21 00:00:00
1940-02-07 00:00:00
1940-11-13 00:00:00
1941-06-20 00:00:00
1941-10-23 00:00:00
1942-08-09 00:00:00
1942-08-24 00:00:00
1943-07-17 00:00:00
1944-12-21 00:00:00
1946-04-20 00:00:00
1946-11-12 00:00:00
1947-09-27 00:00:00
1948-05-27 00:00:00
1948-11-29 00:00:00
1949-10-05 00:00:00
1950-02-15 00:00:00
1950-06-22 00:00:00
1951-07-26 00:00:00
1952-03-13 00:00:00
1953-02-05 00:00:00
1953-08-08 00:00:00
1953-11-10 00:00:00
None
1954-08-17 00:00:00
1954-12-23 00:00:00
1955-05-25 00:00:00
1955-06-22 00:00:00
1955-09-14 00:00:00
1955-12-22 00:00:00
1956-06-08 00:00:00
1956-07-18 00:00:00
1956-11-06 00:00:00
1956-12-20 00:00:00
1957-06-19 00:00:00
1957-08-28 00:00:00
1957-12-25 00:00:00
1958-07-09 00:00:00
1958-08-12 00:00:00
1958-12-25 00:00:00
1959-01-29 00:00:00
1959-03-19 00:00:00
1959-06-24 00:00:00
1959-11-10 00:00:00
1960-01-21 00:00:00
1960-02-24 00:00:00
1960-05-19 00:00:00
1960-08-10 00:00:00
1960-11-01 00:00:00
1960-12-21 00:00:00
1961-01-25 00:0

In [33]:
for movie in movies_data:
    movie['Release date (datetime)']=date_conversion(movie.get('Release date','N/A'))

In [38]:
movies_data[40]

{'title': 'Sleeping Beauty',
 'Directed by': ['Clyde Geronimi (supervising)',
  'Eric Larson',
  'Wolfgang Reitherman',
  'Les Clark'],
 'Produced by': 'Walt Disney',
 'Written by': 'Erdman Penner',
 'Story by': ['Milt Banta',
  'Winston Hibler',
  'Bill Peet',
  'Joe Rinaldi',
  'Ted Sears',
  'Ralph Wright'],
 'Based on': ['Sleeping Beauty', 'by', 'Charles Perrault'],
 'Starring': ['Mary Costa',
  'Bill Shirley',
  'Eleanor Audley',
  'Verna Felton',
  'Barbara Luddy',
  'Barbara Jo Allen',
  'Taylor Holmes',
  'Bill Thompson'],
 'Narrated by': 'Marvin Miller',
 'Music by': "George Bruns (adapted from Tchaikovsky's Sleeping Beauty Ballet )",
 'Edited by': ['Roy M. Brewer Jr.', 'Donald Halliday'],
 'Production company': 'Walt Disney Productions',
 'Distributed by': 'Buena Vista Distribution',
 'Release date': ['January 29, 1959'],
 'Running time': '75 minutes',
 'Country': 'United States',
 'Language': 'English',
 'Budget': '$6 million',
 'Box office': '$51.6 million (United States/Ca

#### Save new data as  `. pickle` formats

In [8]:
import pickle

def save_data_pickle(name,data):
    with open(name,'wb') as f:
        pickle.dump(data,f)    

In [1]:
import pickle

def load_data_pickle(name):
    with open(name,'rb') as f:
        return pickle.load(f)

In [48]:
save_data_pickle('disney_data3.pickle',movies_data)

In [2]:
movies_data2=load_data_pickle('disney_data3.pickle')

### Atach IMDB/Rotten Tomatoes scores

#### Working with API

In [4]:
movies_data2

[{'title': 'Academy Award Review of',
  'Production company': 'Walt Disney Productions',
  'Release date': ['May 19, 1937'],
  'Running time': '41 minutes (74 minutes 1966 release)',
  'Country': 'United States',
  'Language': 'English',
  'Box office': '$45.472',
  'Running time (int)': 41,
  'Budget (float)': None,
  'Box office (float)': 45.472,
  'Release date (datetime)': datetime.datetime(1937, 5, 19, 0, 0)},
 {'title': 'Snow White and the Seven Dwarfs',
  'Directed by': ['David Hand (supervising)',
   'William Cottrell',
   'Wilfred Jackson',
   'Larry Morey',
   'Perce Pearce',
   'Ben Sharpsteen'],
  'Produced by': 'Walt Disney',
  'Written by': ['Ted Sears',
   'Richard Creedon',
   'Otto Englander',
   'Dick Rickard',
   'Earl Hurd',
   'Merrill De Maris',
   'Dorothy Ann Blank',
   'Webb Smith'],
  'Based on': ['Snow White', 'by The', 'Brothers Grimm'],
  'Starring': ['Adriana Caselotti',
   'Lucille La Verne',
   'Harry Stockwell',
   'Roy Atwell',
   'Pinto Colvig',
   'O

In [3]:
import requests
import urllib

In [4]:
api_key='http://www.omdbapi.com/?i=tt3896198&apikey=3e689679'

api_key2='http://www.omdbapi.com/?apikey=[3e689679]&'


def get_ombd_info(title):
    base_url='http://www.omdbapi.com/'
    parameters = {'apikey':'3e689679','t':title}
    params_encoded= urllib.parse.urlencode(parameters)
    base_url='http://www.omdbapi.com/?'
    full_url = base_url + params_encoded
    return requests.get(full_url).json()

def get_rotten_tomato_score(ombd_info):
    ratings=ombd_info.get('Ratings',[])
    
    for rating in ratings:
        if rating['Source'] =='Rotten Tomatoes':
            return rating['Value']

    return None

get_ombd_info('avengers endgame')

#get_rotten_tomato_score(info)

{'Title': 'Avengers: Endgame',
 'Year': '2019',
 'Rated': 'PG-13',
 'Released': '26 Apr 2019',
 'Runtime': '181 min',
 'Genre': 'Action, Adventure, Drama, Sci-Fi',
 'Director': 'Anthony Russo, Joe Russo',
 'Writer': 'Christopher Markus (screenplay by), Stephen McFeely (screenplay by), Stan Lee (based on the Marvel comics by), Jack Kirby (based on the Marvel comics by), Joe Simon (Captain America created by), Jack Kirby (Captain America created by), Steve Englehart (Star-Lord created by), Steve Gan (Star-Lord created by), Bill Mantlo (Rocket Raccoon created by), Keith Giffen (Rocket Raccoon created by), Jim Starlin (Thanos,  Gamora & Drax created by), Stan Lee (Groot created by), Larry Lieber (Groot created by), Jack Kirby (Groot created by), Steve Englehart (Mantis created by), Don Heck (Mantis created by)',
 'Actors': 'Robert Downey Jr., Chris Evans, Mark Ruffalo, Chris Hemsworth',
 'Plot': "After the devastating events of Avengers: Infinity War (2018), the universe is in ruins. With 

In [5]:
for movie in movies_data2:
    title=movie['title']
    ombd_info = get_ombd_info(title)
    movie['imbd'] = ombd_info.get('imdbRating',None)
    movie['metascore']=ombd_info.get('Metascore',None)
    movie['rotten_tomatoes'] = get_rotten_tomato_score(ombd_info)

In [6]:
movies_data2

[{'title': 'Academy Award Review of',
  'Production company': 'Walt Disney Productions',
  'Release date': ['May 19, 1937'],
  'Running time': '41 minutes (74 minutes 1966 release)',
  'Country': 'United States',
  'Language': 'English',
  'Box office': '$45.472',
  'Running time (int)': 41,
  'Budget (float)': None,
  'Box office (float)': 45.472,
  'Release date (datetime)': datetime.datetime(1937, 5, 19, 0, 0),
  'imbd': '7.2',
  'metascore': 'N/A',
  'rotten_tomatoes': None},
 {'title': 'Snow White and the Seven Dwarfs',
  'Directed by': ['David Hand (supervising)',
   'William Cottrell',
   'Wilfred Jackson',
   'Larry Morey',
   'Perce Pearce',
   'Ben Sharpsteen'],
  'Produced by': 'Walt Disney',
  'Written by': ['Ted Sears',
   'Richard Creedon',
   'Otto Englander',
   'Dick Rickard',
   'Earl Hurd',
   'Merrill De Maris',
   'Dorothy Ann Blank',
   'Webb Smith'],
  'Based on': ['Snow White', 'by The', 'Brothers Grimm'],
  'Starring': ['Adriana Caselotti',
   'Lucille La Verne

In [9]:
save_data_pickle('disney_data_final.pickle',movies_data2)

### Saving data as JSON & CSV

In [10]:
movies_data2_copy=[movie.copy() for movie in movies_data2]

In [11]:
for movie in movies_data2_copy:
    current_date=movie['Release date (datetime)']
    if current_date:
        movie['Release date (datetime)'] = current_date.strftime("%B %d, %Y")
    else:
        movie['Release date (datetime)'] = None

In [14]:
import json

def save_data(title, data):
    with open(title,'w',encoding='utf-8') as f:
        json.dump(data,f,ensure_ascii=False, indent=2 )


In [15]:
save_data('disney_data_final.json', movies_data2_copy)

In [18]:
import pandas as pd
df=pd.DataFrame(movies_data2)

In [20]:
df.shape

(443, 37)

In [21]:
df.to_csv('disney_data_final.csv')